In [ ]:
# -*- coding: utf-8 -*-
"""chatbot_backend.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1bhEnmZFSyhcjJdn1VhPvUdAwtTOgHGc6
"""

# chatbot_backend.py

from transformers import (
    CLIPProcessor,
    CLIPModel,
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline
)
from PIL import Image
import torch
import dotenv
import requests


# ----- Configuration -----
PINECONE_API_KEY = dotenv.get_key(".env", "PINECONE_API_KEY")
PERPLEXITY_API_KEY = dotenv.get_key(".env", "PERPLEXITY_API_KEY")
INDEX_NAME = "multimodal"
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Loaded Pinecone key:", bool(PINECONE_API_KEY))


# ----- Pinecone Initialization -----
from pinecone import Pinecone

print(PINECONE_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)


# ----- Load CLIP -----
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


def generate_with_perplexity2(prompt):
    headers = {
        "Authorization": f"Bearer {PERPLEXITY_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "llama-3.1-sonar-large-128k-online",
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful product assistant. Please answer whether we need to return 'text', 'image' or 'both' for the given query in our rag system. You answer should just be these values only ['text','image','both']"
            },
            {
                "role": "user",
                "content": """ Please answer whether we need to return 'text', 'image' or 'both' for the given query in our rag system. You answer should just be these values only ['text','image','both']
                prompt"""
            }
        ],
        "temperature": 0.7,
        "max_tokens": 256
    }
    
    try:
        response = requests.post(
            "https://api.perplexity.ai/chat/completions",
            json=payload,
            headers=headers,
            timeout=10
        )
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except requests.exceptions.HTTPError as e:
        print(f"Perplexity API HTTP Error: {e.response.text}")
        return "Sorry, I couldn't get a response from the AI assistant."
    except Exception as e:
        print(f"Perplexity API error: {str(e)}")
        return "Sorry, there was an error processing your request."

return_flag=generate_with_perplexity2("What are the features of the Samsung Galaxy S21?")



# ----- Add Perplexity API function -----
def generate_with_perplexity(prompt):
    headers = {
        "Authorization": f"Bearer {PERPLEXITY_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "llama-3.1-sonar-large-128k-online",
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful product assistant."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.7,
        "max_tokens": 256
    }
    
    try:
        response = requests.post(
            "https://api.perplexity.ai/chat/completions",
            json=payload,
            headers=headers,
            timeout=10
        )
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except requests.exceptions.HTTPError as e:
        print(f"Perplexity API HTTP Error: {e.response.text}")
        return "Sorry, I couldn't get a response from the AI assistant."
    except Exception as e:
        print(f"Perplexity API error: {str(e)}")
        return "Sorry, there was an error processing your request."

# ----- Embedding Functions -----
def embed_text(text):
    inputs = clip_processor(text=[text], return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        return clip_model.get_text_features(**inputs)[0].cpu().numpy().tolist()

def embed_image(file):
    image = Image.open(file).convert("RGB")
    inputs = clip_processor(images=image, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        return clip_model.get_image_features(**inputs)[0].cpu().numpy().tolist()

# ----- Main Retrieval + Generation Function -----


def query_vector_db(text=None, image=None, return_type="both"):
    return_type=generate_with_perplexity2(text)
    if not text and not image:
        return {
            "answer": "Please provide a text or image input.",
            "image_url": None,
            "retrieved_items": []
        }
    return_type=generate_with_perplexity2(text)
    if return_type not in ['image','text','both']:
        return_type='both'
        
    # 1. Embed the input(s)
    query_vec = None
    query_text = ""
    if text and not image:
        query_vec = embed_text(text)
        query_text = text
    elif image and not text:
        query_vec = embed_image(image)
        query_text = "What is this product?"
    elif text and image:
        # Combine text and image embeddings (optional: use a weighted average or concatenation)
        text_vec = embed_text(text)
        image_vec = embed_image(image)
        query_vec = [(t + i) / 2 for t, i in zip(text_vec, image_vec)]
        query_text = text

    # 2. Determine which vector types to return
    if return_type == "both":
        query_types = ["text", "image"]
    elif return_type in {"text", "image"}:
        query_types = [return_type]
    else:
        raise ValueError("return_type must be one of: 'text', 'image', 'both'")

    # 3. Query Pinecone and build product cards
    retrieved_info = ""
    retrieved_items = []

    for qtype in query_types:
        results = index.query(
            vector=query_vec,
            top_k=5,
            include_metadata=True,
            filter={"type": {"$eq": qtype}}
        )

        for match in results.get("matches", []):
            md = match.get("metadata", {})
            name = md.get("name", "Unknown Product")
            category = md.get("category", "Unknown")
            price = md.get("price", "N/A")
            #product_id = md.get("product_id", "N/A")
            vector_id = match.get("id", "")
            product_id = vector_id.split("_")[0] if "_" in vector_id else "N/A"

            image_type = md.get("type", "unknown")
            img_idx = md.get("img_idx", None)

            retrieved_info += (
                f"- Name: {name}, Category: {category}, Price: {price}, "
                f"ID: {product_id}, Type: {image_type}\n"
            )

            retrieved_items.append({
                "product_id": product_id,
                "title": name,
                "description": f"{category} – ${price}",
                "image_type": image_type,
                "img_idx": img_idx  # Used to fetch the right image from your CSV/URL store
            })

    # 4. Prompt LLM
    prompt = f"""You are a helpful product assistant.

Product Info:
{retrieved_info}

Question:
{query_text}

Answer:"""

    answer = generate_with_perplexity(prompt)

    return {
        "answer": answer,
        "image_url": None,
        "retrieved_items": retrieved_items
    }



preprocessed_df=pd.read_csv("D:\\UCHICAGO\\UChicago Courses\\Generative AI Principles\\Final_Project\\dataset\\preprocessed_data.csv")
product_id_url=queryreturn['retrieved_items'][5]['product_id']
image_url=preprocessed_df[preprocessed_df['Uniq Id']==product_id_url]['Image']